In [ ]:
!pip install git+https://github.com/mathLab/EZyRB
!mkdir data
!wget -P data https://github.com/giovastabile/roma_phd_course_exercises/raw/master/LESSON_1/data/tut1_coord.npy
!wget -P data https://github.com/giovastabile/roma_phd_course_exercises/raw/master/LESSON_1/data/tut1_mu.npy
!wget -P data https://github.com/giovastabile/roma_phd_course_exercises/raw/master/LESSON_1/data/tut1_snapshots.npy
!wget -P data https://github.com/giovastabile/roma_phd_course_exercises/raw/master/LESSON_1/data/tut1_triangles.npy

In [ ]:

import sys
import numpy as np
import matplotlib.tri as mtri
import matplotlib.pyplot as plt
from matplotlib import cm


from ezyrb import POD, RBF, Database
from ezyrb import ReducedOrderModel as ROM
import numpy.linalg as LA
from mpl_toolkits import mplot3d
np.set_printoptions(threshold=sys.maxsize)


def function(x,y):
    return  (x **2) + 3*(y **2) 
    
def dist(p1,p2):
    return np.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)
    
def rad_bas(r):
    c = 0.03
    return np.exp(-(c**2)*(r)**2)


#let's just import the parameter matrix
train_param = np.load('data/tut1_mu.npy')

print("Shape of the parameter matrix: ",train_param.shape)

#let's set the number of sampling points in the parameter space
n_samples = train_param.shape[0]

print("Parameter matrix: ",train_param)



plt.plot(train_param[:,0],train_param[:,1],'*')
#plt.plot(test_param[:,0],test_param[:,1],'r*')
plt.title('Train vs test parameter location')
plt.grid()
plt.show()
# ## Error Approximation & Improvement
# 
# Creating dataset
x = np.outer(np.linspace(-0, 10, 32), np.ones(32))
y = np.outer(np.linspace(-2, 2, 32), np.ones(32))
y = y.T # transpose
z = function(x,y)
print(x.shape)
print(y.shape)
# Creating figure
fig = plt.figure(figsize =(14, 9))
ax = plt.axes(projection ='3d')

 
# Creating plot
surf = ax.plot_surface(x, y, z,cmap=cm.coolwarm,
                       linewidth=0, antialiased=False, alpha=0.4)
fig.colorbar(surf, shrink=0.5, aspect=5)

xp = np.zeros(n_samples)
yp = np.zeros(n_samples)
zp = np.zeros(n_samples)

for i in range(n_samples):
    xp[i] = train_param[i,0]
    yp[i] = train_param[i,1]
    zp[i] = function(xp[i],yp[i])
    
ax.scatter(xp,yp,zp,'*', color='green')

A = np.zeros((n_samples,n_samples))

for i in range(n_samples):
    for j in range(n_samples):
        A[i,j] = rad_bas(dist(train_param[i],train_param[j]))

print(A)

w = LA.inv(A)@zp

print(w)

def rbf_eval(p):
    val=0
    for i in range(n_samples):
        val=val+w[i]*rad_bas(dist(train_param[i],p))
    return val
    
q=np.array([6.4, 0.6])

print(rbf_eval(q))
print(function(q[0],q[1]))
print((rbf_eval(q)-function(q[0],q[1]))/function(q[0],q[1]))

ax.scatter(q[0],q[1],rbf_eval(q),'*',color='red')

z_rbf = np.zeros(z.shape)

for i in range(x.shape[0]):
    for j in range(x.shape[1]):
          z_rbf[i,j] = rbf_eval(np.array([x[i,j],y[i,j]]))

surf2 = ax.plot_surface(x, y, z_rbf,cmap=cm.coolwarm,
                       linewidth=0, antialiased=False, alpha=0.4)
#fig.colorbar(surf, shrink=0.5, aspect=5)

# show plot
plt.show()
